# Class Practice Session

## Goal:
- Read vertex position coordinates and texture coordinates of shape from .obj file.
- Use a perspective camera
- Render textured shape object.
- Rotate the camera around the vertical axis.

## Change from Assignment 5
- No more clockface with a rotating second hand with diamond shape at the ends.
- The 3D object data with position and texture coordinates will be read from a .obj file.
    - NOTE:
    - You must download LoadObject.py to your current work folder.
    - Import getObjectData from LoadObject package
        - ``` from LoadObject import getObjectData ```
    - getObjectData usage: ```getObjectData(<obj file>, normal=True/False, texture=True/False)```
- ctx.vertex_array changes for the shader program to accept multiple attributes (specifically 3D position coordinates and 2D texture coordinates) will be as follows: ```ctx.vertex_array(program,[(<buffer>, "3f 2f", "position", "uv")])```
- Instead of clock face texture, a brick texture will be read and applied to the teapot.
- A perspective orbiting Camera will be used.
- If it is a real 3D object, then enable Depth Test to remove parts hidden from the view.
    - How to enable Depth Test: ``` gl.enable(gl.DEPTH_BUFFER) ```

## Imports

In [1]:
import pygame
import moderngl
import numpy
import glm
from LoadObject import getObjectData # This is the new addition.

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
getObjectData("quadFace.obj")
getObjectData("quadFace.obj", texture=True)

geomData = [
    -1., -1., 0.,   0., 0.,
    1., -1., 0.,    1., 0.,
    1., 1., 0.,     1., 1.,
    -1., -1., 0.,   0., 0.,
    1., 1., 0.,     1., 1.,
    -1., 1., 0.,    0., 1.

]

Vertex position Count:  4 (-1.0, -1.0, 0.0)
Vertex texture coordinate Count:  4 (0.0, 0.0, 0.0)
Face count:  2.0
Vertex position Count:  4 (-1.0, -1.0, 0.0)
Vertex texture coordinate Count:  4 (0.0, 0.0, 0.0)
Face count:  2.0
texture exists


## Intialize

In [3]:
pygame.init() # Initlizes its different modules. Display module is one of them.

width = 840
height = 480
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE) 
pygame.display.set_mode((width, height), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title = "Class Practice: Instructor")
gl = moderngl.get_context() # Get Previously created context.
gl.info["GL_VERSION"]

'4.6.0 NVIDIA 560.94'

### Get the geometry data load it to a GPU buffer

In [4]:
diamond_position_data = [
    0.0, 0.8,
    -0.8, 0,
    0.8, 0,
    0.8, 0,
    -0.8, 0,
    0, -0.8
]
diamond_geometry = numpy.array(diamond_position_data).astype("float32")
#Push the Geometry Data to the GPU buffer.
diamond_geometry_buffer = gl.buffer(diamond_geometry) # Create a Vertex buffer object from the data

## Create shader program(s) and Renderables

#### Vertex Shaders
The Vertex shader must set the value of a predefined vec4 variable gl_Position.

In [5]:
#
# Vertex shader(s)
#

diamond_vertex_shader_code = '''
#version 330 core

// add attributes variables here
layout (location=0) in vec2 position;

uniform mat2 M;
uniform float scale;
uniform float radius;

void main() {
    vec2 displacement = radius * M * vec2(0,1);
    vec2 P = M * (scale * position) + displacement;
    gl_Position = vec4(P,0.0, 1.0);
}
'''
line_vertex_shader_code = '''
#version 330 core

// add attributes variables here
uniform mat2 M;
uniform float radius;

void main() {
    vec2 P = vec2(0);
    if (gl_VertexID > 0){
        P = radius * M * vec2(0,1);
    }
    gl_Position = vec4(P,0.0, 1.0);
}
'''
face_vertex_shader_code = '''
#version 330 core

layout (location=0) in vec3 position; 
layout (location=1) in vec2 uv; 

out vec2 f_uv; // Texture coordinate
uniform mat2 M;
void main() {
    vec2 P, uv;
    switch (gl_VertexID){
        case 0: 
        case 3:
            uv = vec2(0,0);
            P = vec2(-1,-1);
            break;
        case 1: 
            uv = vec2(1,0);
            P = vec2(1,-1);
            break;
        case 2:
        case 4:
            uv = vec2(1,1);
            P = vec2(1,1);
            break;
        case 5:
            uv = vec2(0,1);
            P = vec2(-1,1);
            break;
    }
    f_uv = uv;
    gl_Position = vec4(M*P,0.0, 1.0);
}
'''

#
# Fragment shader(s)
#

diamond_fragment_shader_code = '''
#version 330 core
// Add output variable here
out vec4 color;
uniform vec3 inColor;
void main() {
    //set output color value here
    color = vec4(inColor,1);
}
'''
face_fragment_shader_code = '''
#version 330 core
in vec2 f_uv;
uniform sampler2D map;
// Add output variable here
out vec4 color;

void main() {
    //set output color value here
    color = texture(map, f_uv);
}
'''
#
# Programs
#

diamond_program = gl.program(
    vertex_shader= diamond_vertex_shader_code,
    fragment_shader= diamond_fragment_shader_code
)

line_program = gl.program(
    vertex_shader= line_vertex_shader_code,
    fragment_shader= diamond_fragment_shader_code
)

face_program = gl.program(
    vertex_shader= face_vertex_shader_code,
    fragment_shader= face_fragment_shader_code
)

#
# Renderables
#
diamond_renderable = gl.vertex_array(diamond_program,
    [( diamond_geometry_buffer, "2f", "position" ) ]
)

line_renderable = gl.vertex_array(line_program,
    []
)

face_renderable = gl.vertex_array(face_program,
    []
)

##  Create Texture object and specify filter and wrap options.
__Load image and get the data in a sequence of bytes__  
```pygame.image.load()``` will return a Suface object that has image data.  
```texture_img = pygame.image.load(<image_file>)```  
```texture_data = pygame.image.tobytes(texture_img,channels,True)``` \#tobytes(Surface, format=\<"RGB" or "RGBA"\>, \<to Flip or no to flip True/False\>) -> bytes    
\#w, h = texture_img.get_size()   

__Create Texture object and send the data__  
```ctx.texture(size = <size tuple>, data=<texture data in bytes>,components=<Number of color channels>)``` \# returns texture object filled with data

__Create a sampler object that associates filter and wrap options with the texture__  
```ctx.sampler(texture=<texture object>, filter=(<Minification filter>, <magnification filter>), repeat_x=<True/False>, repeat_y=<True/False>)``` \# returns a sampler object

LINEAR_MIPMAP_LINEAR in minification filtering requires that you generate MipMap of the texture a priori. 
```<texture object>.build_mipmaps()```
The sampler object must be attached to a texture unit before it can be used in the shader program.  
```<sampler object>.use(n)```

In [6]:
texture_img = pygame.image.load("clockFace.png") 
texture_data = pygame.image.tobytes(texture_img,"RGB", True) 
faceTexture = gl.texture(texture_img.get_size(), data = texture_data, components=3)
faceSampler = gl.sampler(texture=faceTexture, filter=(gl.LINEAR, gl.LINEAR)) 

FileNotFoundError: No file 'clockFace.png' found in working directory 'c:\Users\caitl\OneDrive\Desktop\graphics-class\class\92525'.

## Define Camera Parameters here
Camera parameters for view transformation matrix computation are: ``` eye_point, lookat_point or target_point, up_vector ```
- Use glm.lookat to compute the view transformation matrix
    - glm.lookAt usage: ``` glm.lookAt(eye_point, target_point, up_vector)```

Camera parameters for perspective transformation matrix computation are: ``` fov (in radians), aspect_ratio, near and far``` plane distances.
- Use glm.perspective to compute the perspective transformation Matrix.
    - glm.perspective usage: ``` glm.perspective(fov_radian, aspect, near, far) ```
 
You may set uniform matrix values from the glm computed 4x4 matrix as:
``` program[<uniform matrix name>].write(<matrix>) ```

## Render loop

In [ ]:
running = True
clock = pygame.time.Clock()
alpha = 0

while running:   
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
    # event.key == 27 means Escape key is pressed.
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == 27):
            running = False
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
    aspect = width/height
    # create the aspect ratio correction matrix
    
    M_aspect = glm.mat2(1/aspect, 0, 0, 1) if aspect > 1 else glm.mat2(1, 0, 0, aspect)

    #Create the rotation matrix
    #  Let it be M_rotate
    # Remember positive alpha values are for anticlockwise rotation. For clockwise rotation, use -alpha
    # M = M_aspect * M_rotate
    # ... Part of assignment 4 and 5
    
    M = M_aspect
    
    ### Add render code below
    # Clear the display window with the specified R, G, B values using function ctx.clear(R, G, B)
    gl.clear(0.5,0.5,0.0)
    
    # Make one or more Render Calls to instruct the GPU to render by executing the shader program with the provided data.
    faceSampler.use(0)
    face_program["map"] = 0
    face_program["M"].write(M)
    face_renderable.render(moderngl.TRIANGLES, vertices=6)
    
    diamond_program["M"].write(M)
    
    diamond_program["scale"] = 0.1
    diamond_program["inColor"] = 1, 1, 0
    
    diamond_program["radius"] = 0.8
    diamond_renderable.render()
    
    diamond_program["radius"] = 0.0
    diamond_renderable.render()
    
    line_program["M"].write(M)
    line_program["radius"] = 0.8
    line_program["inColor"] = 0, 0, 0
    line_renderable.render(moderngl.LINES, vertices=2)  
    
    pygame.display.flip()
    clock.tick(60)  # limits FPS to 10
    alpha = alpha + 1
    if alpha > 360:
        alpha = 0

pygame.display.quit()